[https://dados.gov.br/swagger-ui/index.html#/]

In [108]:
import os
import json
import requests

from dotenv import load_dotenv
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlretrieve

If you need debugging

```python
import logging
import http.client

http.client.HTTPConnection.debuglevel = 10

# You must initialize logging, otherwise you'll not see debug output.
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True
```

In [37]:
data_directory_path = "./data"
env_filepath = ".env"

In [3]:
load_dotenv(env_filepath)

True

In [4]:
request_headers = {
    'accept': 'application/json',
    'chave-api-dados-abertos': os.environ["CHAVE_API_DADOS_GOV_BR"],
}

request_base_url = urlparse('https://dados.gov.br/dados/api/publico/')

#### Requesting the list of organizations

In [5]:
organizations = []
pagina = 0
while True:
    pagina += 1
    print(pagina, end=', ')
    
    request_params = {
        'pagina' : pagina,
    }
    
    requested = requests.get(
        urljoin(request_base_url.geturl(),'organizacao'), 
        headers=request_headers, 
        params=request_params
    )
    
    if requested.json():
        organizations.extend(requested.json())
    else:
        break


1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 

#### Requesting details of each organization

In [6]:
o = 0
for org in organizations:    
    o += 1
    requested = requests.get(
        urljoin(request_base_url.geturl(),'/'.join(['organizacao',org['id']])), 
        headers=request_headers, 
        params=request_params
    )
    org.update(requested.json())
    print(o, end=', ' if o % 20 else '\n')

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40
41, 42, 43, 44, 45, 46, 47, 48, 49, 

KeyboardInterrupt: 

#### Dumping organizations data to a json file

In [102]:
with open(os.path.join(data_directory_path,'organizations_tmp.json'),'w+') as f:
    json.dump(organizations,f)

#### Loading organizations from json file

In [72]:
with open(os.path.join(data_directory_path,'organizations.json'),'r') as f:
    organizations = json.load(f)

#### Filtering the organizations to our needs

In [81]:
organizations = [o for o in organizations if 'universidade' in o['nome'].lower()]

In [71]:
organizations = [o for o in organizations if 'UFRN' in o['titulo']]

#### Requesting the list of datasets from each organization

In [84]:
o = 0 
for org in organizations:
    o += 1
    if org.get('conjuntoDados'): continue
    if not int(org['qtdConjuntoDeDados']): continue
    print(o,org['titulo'], end=': ')

    conjunto_dados = []
    pagina = 0
    while True:
        pagina += 1
        print(pagina, end=', ')
        request_params = {
            'idOrganizacao' : org['id'],
            'pagina' : pagina,
        }
        
        request_url = "https://dados.gov.br/dados/api/publico/conjuntos-dados"
        requested = requests.get(request_url, headers=request_headers, params=request_params)
    
        if requested.json():
            conjunto_dados.extend(requested.json())
        else:
            break
    org['conjuntoDados'] = conjunto_dados
    print('!')

43 Universidade Federal de Viçosa: 1, 2, 3, 4, !
44 Universidade Federal do Rio Grande do Norte - UFRN: 1, 2, 3, 4, 5, 6, 7, !
45 Hospital Universitário Walter Cantídio da Universidade Federal do Ceará (CH-UFC): 1, 2, 3, !
46 Universidade Federal de Santa Maria - UFSM: 1, 2, 3, !
47 Universidade Federal de São João del-Rei - UFSJ: 1, 2, !
48 Universidade Federal de Mato Grosso do Sul - UFMS: 1, 2, 3, 4, !
49 Universidade Federal de Ciências da Saúde de Porto Alegre - UFCSPA: 1, 2, 3, 4, !
50 Universidade Federal da Fronteira Sul: 1, 2, !
51 Universidade Federal de Pelotas - UFPel: 1, 2, 3, 4, !
52 Fundação Universidade Federal de Sergipe - UFS: 1, 2, !
53 Hospital Universitário da Universidade Federal do Piauí - HU-UFPI: 1, 2, 3, !
54 Universidade Federal de São Carlos - UFSCar: 1, 2, !
55 Universidade Federal do Maranhão - UFMA: 1, 2, 3, !
56 Universidade Federal Rural de Pernambuco - UFRPE: 1, 2, 3, !
57 Universidade Federal do Amazonas - UFAM: 1, 2, 3, !
58 Fundação Universidade Fed

#### Requesting dataset details

In [100]:
for o, org in enumerate(organizations):
    org_directory_name = org['nome'].replace(' ','_').lower().replace('/','_').replace('\\','_').strip()
    org_directory_path = os.path.join(data_directory_path,org_directory_name)
    org_json_filepath = os.path.join(data_directory_path,org_directory_name+'.json')

    if os.path.exists(org_json_filepath): continue
    if not int(org['qtdConjuntoDeDados']): continue

    print(o, org['titulo'], end=':\n')
    request_params = {
        
    }
    for d, dado in enumerate(org['conjuntoDados']):
        print('\t'+str(d), end=' (')
        request_url = "https://dados.gov.br/dados/api/publico/conjuntos-dados/" + dado['id']
        response = requests.get(request_url, headers=request_headers, params=request_params)
        response_json = response.json()
        if response_json:
            print(dado['title'], end=')\n')
            dado.update(response_json)        
    
    
    with open(org_json_filepath,'w+') as f:
        json.dump(org,f)

#### Retrieving dataset resource

In [104]:
organizations_jsons = [f for f in os.listdir(data_directory_path) if f.endswith('.json') and f != 'organizations.json']

In [137]:
organizations_jsons = [json for json in organizations_jsons if 'ufrn' in json]
organizations_jsons

['universidade_federal_do_rio_grande_do_norte_-_ufrn.json']

In [136]:
for o,organization_json_filepath in enumerate(organizations_jsons):
    with open(os.path.join(data_directory,organization_json_filepath),'r') as f:
        organization_json = json.load(f)
    organization_directory_name = organization_json_filepath[:-5]
    organization_directory_path = os.path.join(data_directory_path,organization_directory_name)

    os.makedirs(organization_directory_path, exist_ok=True)

    print(organization_directory_name)

    for conjunto in organization_json['conjuntoDados']:
        print('\t',conjunto['title'])
        for r,recurso in enumerate(conjunto['recursos']):
            if not recurso['formato']: continue
            url = urlparse(recurso['link'])
            filename = url.path.split('/')[-1]
            filepath = os.path.join(organization_directory_path,filename)
            if os.path.exists(filepath): continue
            print('\t\t',r, recurso['titulo'],filename)
            urlretrieve(url.geturl(),filepath)


universidade_federal_do_rio_grande_do_norte_-_ufrn
	 I Hackathon UFRN
	 Relação Empenhos COVID inscritos em RPNP
	 Acervo aquisição
	 Servidores Aposentados
	 Trabalhos de Conclusão de Curso
	 Requisição de Serviço para Licitação
	 Cursos de Graduação
	 Requisição de Parecer Técnico/Avaliação
	 Bolsas de Apoio
	 Requisição de Material para Importação
	 Requisição de Serviço para Pessoa Jurídica
	 Requisição de Reserva de Veículo
	 Requisição de Meio Ambiente
	 Requisição de Materiais Informacionais do Contrato
	 Requisição de Atividade de Campo
	 Requisição de Devolução de Material
	 Grupos de Pesquisa
		 0 Grupos de Pesquisa grupos-de-pesquisa.csv
		 1 Dicionário de Dados - Grupos de Pesquisa dicionario-de-dados---grupos-de-pesquisa.pdf
	 Solicitação de Material de Registro de Preço
		 0 Solicitações de Materiais de Registro de Preço req-material-registro-preco.csv
		 1 Dicionário de Dados - Solicitações de Materiais de Registro de Preço dicionario-de-dados---solicitacao-de-material